# Zoptymalizowana symulacja plazmowa Particle in Cell w Pythonie

## Dominik Stańczak, 2016

### Plazma - po co ją symulować?
* *mission-driven* w sensie zastosowań do fuzji termojądrowej
    * Schemat fuzji DT
    * Potrzebujemy ogromnych temperatur, by to osiągnąć - osiągalne jedynie w plazmach

* Super złożony układ: nie znamy analitycznych rozwiązań na praktycznie żaden z ciekawszych przypadków
* Główny koncepcyjny problem: liczba dalekozasięgowych oddziaływań w układzie rośnie jak $n^2$ w liczbie cząstek

### Plazma - jak ją symulować?

* Równanie Vlasova
    * Paskudne, bo w jednej chwili czasu pracujemy na 6-wymiarowej siatce. Więc problem skaluje się jak $(N_x)^3(N_v)^3$...
* Przybliżenia - MHD
* Oraz są modele PIC...

### Modele Particle in Cell

### Główny algo
Myk jest koncepcyjnie prosty. To co nas boli, to obliczanie sił - złożoność $O(n^2)$.

Więc po co liczyć siły?

Zdefiniujmy sobie dyskretną siatkę (*grid*) o liczbie komórek $n_g << n$. Każda z cząstek wpada w którąś komórkę.

Policzmy, ile jest cząstek każdego rodzaju w każdej komórce. Sumując ładunek, dostajemy gęstość ładunku. Złożoność $O(n)$ - wykonujemy tą operację raz dla każdej cząstki. To tzw. *charge deposition*.

Mamy więc na $n_g$ komórek siatki zdefiniowaną gęstość ładunku. Możemy teraz wykorzystać równanie Poissona:

$$\rho = \nabla E = -\Delta V$$

Policzyć to - ze złożonością zazwyczaj kwadratową (spectral solver przez FFT), ale w $n_g$ - zaś jeśli $n_g << n$, to $n_g^2 << n^2$!

Mamy więc pola na siatce. Teraz możemy zinterpolować z siatki do cząstek, raz na cząstkę - $O(n)$, wciąż mniej niż kwadratowo!

A teraz wykorzystujemy pola w każdej cząstce, żeby je *popchnąć* do przodu (*particle pusher*) (złożoność liniowa!)

I wracamy do punktu wyjścia!

# Optymalizacja kodu

Jak zrobić, żeby kod biegał szybko?
* Python jest wolny - ale C i Fortran nie są.
    * Numpy
    * Numba
Pracka re: hpc python z hydrodynamiką
* Paralelizm
    * Problemy trywialnie paralelizowalne
    * Złożone podejścia do paralelizmu
    * Karty graficzne!
    * Kopiowanie pamięci

### Triki i myki

* Jak uwzględnić prąd? Bardzo łatwo, depozytować $q\vec{v}$!

* Pole magnetyczne - wymaga aby symulacja była $1D-3V$ 

* Solver pola relatywistycznego jest lokalny, niefourierowski (prędkość rozchodzenia się fali)

* Koncepcyjnie łatwo to wszystko wstawić na kartę graficzną!